In [2]:
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm
# from transformers import AutoTokenizer

In [7]:
train_df = pd.read_csv("./data/train_back_translation.csv")

* 유의어로 교체(Synonym Replacement, SR): 문장에서 랜덤으로 stop words가 아닌 n 개의 단어들을 선택해 임의로 선택한 동의어들 중 하나로 바꾸는 기법.

* 랜덤 삽입(Random Insertion, RI): 문장 내에서 stop word를 제외한 나머지 단어들 중에서, 랜덤으로 선택한 단어의 동의어를 임의로 정한다. 그리고 동의어를 문장 내 임의의 자리에 넣는걸 n번 반복한다.

* 랜덤 교체(Random Swap, RS): 무작위로 문장 내에서 두 단어를 선택하고 위치를 바꾼다. 이것도 n번 반복

* 랜덤 삭제(Random Deletion, RD): 확률 p를 통해 문장 내에 있는 각 단어들을 랜덤하게 삭제한다.

In [4]:
# EDA 사용방법
from koeda import EDA

eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.2, prob_rd=0.1
)

text = "아버지가 방에 들어가신다"

result = eda(text)
print(result)

내부가 방에 들어가신다


In [5]:
# AEDA 사용방법
from koeda import AEDA

aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)

어머니가 : 집 , 을 나가신다


sentence_1을 바꿀지, sentence_2를 바꿀지, sentence_1,2 모두 바꿀지? <br/>
5점 경우는 2700개의 데이터를 추가로 만들고 싶다 <br/>
문장은 사이클로 돌리고 sent 1,2,(1,2)는 random으로 선택하자

In [8]:
train_df = get_preprocessed_label(train_df)
train_df

,Unnamed: 0,id,source,sentence_1,sentence_2,label,binary-label,label-binning,preprocessed_label
0,0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2.0~2.9,2.0
1,1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4.0~4.9,4.0
2,2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2.0~2.9,2.0
3,3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3.0~3.9,3.0
4,4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10219,95,boostcamp-sts-v1-train-5160,slack-sampled,한국에서 만나요!!,한국에서 만나요,4.5,1.0,4.0~4.9,4.0
10220,96,boostcamp-sts-v1-train-497,slack-sampled,!!! 당신은 몸이 좋습니까?,!!!당신은몸이좋습니까?,4.5,1.0,4.0~4.9,4.0
10221,97,boostcamp-sts-v1-train-3999,nsmc-sampled,와.. 로그인한지 좀 됐네요.,와 로그인한지 좀 됐네요,4.5,1.0,4.0~4.9,4.0
10222,98,boostcamp-sts-v1-train-8774,nsmc-sampled,그나저나..강풀 원작의 내용을 영화로 번역하기 힘든가요?,그나저나..강풀원작의내용을영화로번역하기힘든가요?,4.5,1.0,4.0~4.9,4.0


In [9]:
train_df['preprocessed_label'].value_counts().sort_index()

0.0    2981
1.0    1658
2.0    1075
3.0    1334
4.0    2245
5.0     931
Name: preprocessed_label, dtype: int64

In [12]:
from koeda import AEDA
import pandas as pd
import numpy as np
import copy


def get_preprocessed_label(df):
    """_summary_

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    for i in range(len(df)):
        df.loc[i, "preprocessed_label"] = round(df.loc[i, "label"])

    return df


def concat_AEDA_sent(df_label, aug_nums):
    """라벨 x만 있는 데이터프레임에 aug_nums 만큼 AEAD된 데이터를 추가

    Args:
        df_label (DataFrame): 라벨 x만 있는 데이터프레임
        aug_nums (int): 증강하고자 하는 수
    Returns:
        AEDA가 추가된 라벨 x의 데이터프레임
    """
    np.random.seed(12)
    aug_idx = np.random.randint(0, 3, size=aug_nums)
    dataset_idx = 0
    aug_df_label = copy.deepcopy(df_label)

    aeda = AEDA(
        morpheme_analyzer="Okt",
        punc_ratio=0.3,
        punctuations=[".", ",", "!", "?", ";", ":"],
    )

    for i in range(aug_nums):
        if dataset_idx >= len(df_label):
            dataset_idx = 0

        origin_id = df_label.iloc[dataset_idx]["id"]
        origin_source = df_label.iloc[dataset_idx]["source"]
        origin_sentence_1 = df_label.iloc[dataset_idx]["sentence_1"]
        origin_sentence_2 = df_label.iloc[dataset_idx]["sentence_2"]
        origin_label = df_label.iloc[dataset_idx]["label"]
        origin_binarylabel = df_label.iloc[dataset_idx]["binary-label"]
        origin_preprocessed_label = df_label.iloc[dataset_idx]["preprocessed_label"]

        if aug_idx[i] == 0:  # sent_1만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_1"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent],
                    "sentence_2": [origin_sentence_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        elif aug_idx[i] == 1:  # sent_2만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [origin_sentence_1],
                    "sentence_2": [aug_sent],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        else:  # sent_1과 2를 모두 aug
            sent_1 = aug_df_label.iloc[dataset_idx]["sentence_1"]
            sent_2 = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent_1 = aeda(sent_1)
            aug_sent_2 = aeda(sent_2)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent_1],
                    "sentence_2": [aug_sent_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        dataset_idx += 1

    return aug_df_label


def AEDA_data():

    train = pd.read_csv("./data/train_back_translation.csv")
    train = get_preprocessed_label(train)

    train_0 = train[train["preprocessed_label"] == 0].reset_index(drop=True)
    train_1 = train[train["preprocessed_label"] == 1].reset_index(drop=True)
    train_2 = train[train["preprocessed_label"] == 2].reset_index(drop=True)
    train_3 = train[train["preprocessed_label"] == 3].reset_index(drop=True)
    train_4 = train[train["preprocessed_label"] == 4].reset_index(drop=True)
    train_5 = train[train["preprocessed_label"] == 5].reset_index(drop=True)
    
    train_1_aug = concat_AEDA_sent(train_1, 2981 - 1658)
    train_2_aug = concat_AEDA_sent(train_2, 2981 - 1075)
    train_3_aug = concat_AEDA_sent(train_3, 2981 - 1334)
    train_4_aug = concat_AEDA_sent(train_4, 2981 - 2245)
    train_5_aug = concat_AEDA_sent(train_5, 2981 - 931)

    train_0 =  pd.concat([train_0, train_1_aug, train_2_aug, train_3_aug, train_4_aug, train_5_aug], ignore_index=True)
    
    auged_df = train_0.sample(frac=1).reset_index(drop=True)
    
    return auged_df

In [13]:
final_df = AEDA_data()

In [14]:
final_df.to_csv('./train_auged.csv', index=False)